In [14]:
from datasets import load_dataset, DatasetDict
dataset = load_dataset("json", data_files="../train.json",split="train")

print(dataset)

Generating train split: 8 examples [00:00, 1224.75 examples/s]

Dataset({
    features: ['id', 'prompt', 'response', 'reward'],
    num_rows: 8
})


In [15]:
dataset_dict = DatasetDict({
    "train": dataset
})
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'response', 'reward'],
        num_rows: 8
    })
})


In [19]:
import os
import shutil
if os.path.exists("../aixue_test_data"):
    shutil.rmtree("../aixue_test_data")

In [20]:
os.makedirs("../aixue_test_data", exist_ok=True)
dataset_dict.save_to_disk(
    dataset_dict_path="../aixue_test_data",
    max_shard_size="500MB",  # 可选：分片大小控制
    num_proc=1,               # 可选：并行进程数
)

Saving the dataset (1/1 shards): 100%|██████████| 8/8 [00:00<00:00, 1857.32 examples/s]


# Response Test

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "/root/group-shared/models/base_models/Qwen3-32B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 17/17 [00:48<00:00,  2.86s/it]


In [13]:
# prepare the model input
prompt = """你是一名专注于1对1自然拼读教学的英语老师。

# 教学目标
## 核心课程目标：
    1. 建立“字母/字母组合”与“发音”的牢固对应关系（音形联结），掌握单个字母基础发音及常见字母组合发音。
    2. 发展学生语音意识（音素、音节、重音感知），提升单词记忆与拼写能力。
    3. 最终目标：实现“见词能读，听音能写”。
## 单节课教学目标：
    1. 个性化教学：根据学生能力调整内容呈现和练习方式，激发兴趣，提升效果。
    2. 保持专注与信心：控制单次学习时长，避免连续错误过多，减少疲劳感和挫败感。

# 学生画像
    1. 年龄：7岁
    2. 性别：女
    3. 所在地：中国三线城市
    4. 英语基础：
       - 掌握26个英文字母名称。
       - 仅会极少量简单会话（如：What's your name?）。

# 当前教学状态
    1. 课节内容：教授字母 A、B、C 的发音（a: /æ/, b: /b/, c: /k/）。
    2. 当前**环节**：字母 A (/æ/) 的发音练习。
    3. 主题关联：教学围绕'苹果 (apple)'展开，练习部分如有单词，建议与之相关。

# 教学工具箱 (可选学习范式)
## 字母教学包含4种基础练习类型（难度递增）, 例如对于字母a：
1. 纯音素重复：`/æ/ /æ/ /æ/` (重复发音3次)
2. 音形对应：`a says /æ/` (建立字母与发音关联)
3. 音素-单词关联：`/æ/ /æ/ apple` (强化发音在单词中的感知)
4. 综合练习：`a says /æ/, /æ/ /æ/ apple` (整合字母、发音与单词)
## 智能纠错策略 (根据错误类型选择下一步)
1. 错误类型A (字母名称错，发音对)：例如学生说'a says /æ/' (a读错，/æ/正确)。  
    **下一步：** 聚焦字母名称练习。老师示范：'a' (仅字母名称)。
2. 错误类型B (单词发音错，字母发音对)：例如学生说'/æ/ /æ/ apple' (apple发音错，/æ/正确)。  
    **下一步：** 聚焦目标单词练习。老师示范：'apple' (仅单词)。
## 教学控制参数
1. 单字母(单环节)最大教学次数：4次 (若学生能力强，`综合练习`一次性通过，可减少次数, 直接进入下一个**环节**)。
2. 单次跟读最大重复次数：1-2次 (避免疲劳)。
3. 核心原则：及时强化正确，精准纠正错误，保持学习动力。

# 学生学习记录
1. 学习次数：1 次
2. 上次练习内容：'/æ/ /æ/ apple'
3. 上次表现评分：B (部分正确)
4. 具体错误：单词'apple'中的辅音'/p/'发音有瑕疵。

# 你的任务：制定下一步教学指令
1. **评估：** 基于教学目标、学生基础、当前环节、可选范式、纠错策略、历史表现及教学控制参数，决定下一步的教学。
2. 输出格式要求：
   - 如果结束当前字母教学，直接回复 <END>
   - 如果需要继续学习，直接回复 下一步跟读的句子"""
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
for _ in range(8):
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=32768,
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 
    
    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0
    
    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
    
    print("thinking content:", thinking_content)
    print("content:", content)

thinking content: 
content: 下一步跟读的句子：**apple**
thinking content: 
content: 下一步跟读的句子：**apple**
thinking content: 
content: 下一步跟读的句子：  
**apple**
thinking content: 
content: 下一步跟读的句子：  
**apple**
thinking content: 
content: 下一步跟读的句子：  
**"apple"**  

（根据学生学习记录，她在单词“apple”中的辅音/p/发音有瑕疵，因此本次练习聚焦单词发音的精准性，采用纠错策略中的“错误类型B”，直接强化目标单词的正确发音。）
thinking content: 
content: 下一步跟读的句子：**apple** (仅单词，聚焦单词发音练习，纠正辅音 /p/)
thinking content: 
content: 下一步跟读的句子：  
**"apple"** (仅单词，聚焦单词发音练习，纠正辅音 /p/ 的发音瑕疵)
thinking content: 
content: 下一步跟读的句子：**apple**


In [7]:
# prepare the model input
prompt = """你是一名专注于1对1自然拼读教学的英语老师。

# 教学目标
## 核心课程目标：
    1. 建立“字母/字母组合”与“发音”的牢固对应关系（音形联结），掌握单个字母基础发音及常见字母组合发音。
    2. 发展学生语音意识（音素、音节、重音感知），提升单词记忆与拼写能力。
    3. 最终目标：实现“见词能读，听音能写”。
## 单节课教学目标：
    1. 个性化教学：根据学生能力调整内容呈现和练习方式，激发兴趣，提升效果。
    2. 保持专注与信心：控制单次学习时长，避免连续错误过多，减少疲劳感和挫败感。

# 学生画像
    1. 年龄：7岁
    2. 性别：女
    3. 所在地：中国三线城市
    4. 英语基础：
       - 掌握26个英文字母名称。
       - 仅会极少量简单会话（如：What's your name?）。

# 当前教学状态
    1. 课节内容：教授字母 A、B、C 的发音（a: /æ/, b: /b/, c: /k/）。
    2. 当前**环节**：字母 A (/æ/) 的发音练习。
    3. 主题关联：教学围绕'苹果 (apple)'展开，练习部分如有单词，建议与之相关。

# 教学工具箱 (可选学习范式)
## 字母教学包含4种基础练习类型（难度递增）, 例如对于字母a：
1. 纯音素重复：`/æ/ /æ/ /æ/` (重复发音3次)
2. 音形对应：`a says /æ/` (建立字母与发音关联)
3. 音素-单词关联：`/æ/ /æ/ apple` (强化发音在单词中的感知)
4. 综合练习：`a says /æ/, /æ/ /æ/ apple` (整合字母、发音与单词)
## 智能纠错策略 (根据错误类型选择下一步)
1. 错误类型A (字母名称错，发音对)：例如学生说'a says /æ/' (a读错，/æ/正确)。  
    **下一步：** 聚焦字母名称练习。老师示范：'a' (仅字母名称)。
2. 错误类型B (单词发音错，字母发音对)：例如学生说'/æ/ /æ/ apple' (apple发音错，/æ/正确)。  
    **下一步：** 聚焦目标单词练习。老师示范：'apple' (仅单词)。
## 教学控制参数
1. 单字母(单环节)最大教学次数：4次 (若学生能力强，`综合练习`一次性通过，可减少次数, 直接进入下一个**环节**)。
2. 单次跟读最大重复次数：1-2次 (避免疲劳)。
3. 核心原则：及时强化正确，精准纠正错误，保持学习动力。

# 学生学习记录
1. 学习次数：1 次
2. 上次练习内容：'/æ/ /æ/ apple'
3. 上次表现评分：B (部分正确)
4. 具体错误：单词'apple'中的辅音'/p/'发音有瑕疵。

# 你的任务：制定下一步教学指令
1. **评估：** 基于教学目标、学生基础、当前环节、可选范式、纠错策略、历史表现及教学控制参数，决定下一步的教学。
2. 输出格式要求：
   - 如果结束当前字母教学，直接回复 <END>
   - 如果需要继续学习，直接回复 下一步跟读的句子"""
messages = [
    {"role": "user", "content": prompt}
]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.modeling_utils import load_sharded_checkpoint

model_name = "/root/group-shared/models/base_models/Qwen3-32B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_my= AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

checkpoint_path = "/root/group-shared/jrc/ppo-test/models/4gpu_aixue_data_test/checkpoint-1-out"

"""
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

model_my = AutoModelForCausalLM.from_pretrained(
    checkpoint_path
)
"""
load_sharded_checkpoint(
    model_my,
    checkpoint_path
)

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)
model_my_inputs = tokenizer([text], return_tensors="pt").to(model_my.device)

for _ in range(8):
    # conduct text completion
    generated_ids = model_my.generate(
        **model_my_inputs,
        max_new_tokens=32768
    )
    output_ids = generated_ids[0][len(model_my_inputs.input_ids[0]):].tolist() 
    
    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0
    
    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
    
    print("thinking content:", thinking_content)
    print("content:", content)

Loading checkpoint shards: 100%|██████████| 17/17 [00:40<00:00,  2.40s/it]


thinking content: 
content: 下一步跟读的句子：**a says /æ/**


In [11]:
for _ in range(8):
    # conduct text completion
    generated_ids = model_my.generate(
        **model_my_inputs,
        max_new_tokens=32768
    )
    output_ids = generated_ids[0][len(model_my_inputs.input_ids[0]):].tolist() 
    
    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0
    
    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
    
    print("thinking content:", thinking_content)
    print("content:", content)

thinking content: 
content: a says /æ/, /æ/ /æ/ apple
thinking content: 
content: a says /æ/, /æ/ /æ/ apple
thinking content: 
content: a says /æ/, /æ/ /æ/ apple
thinking content: 
content: a says /æ/, /æ/ /æ/ apple
thinking content: 
content: 下一步跟读的句子：**a says /æ/, /æ/ /æ/ apple**
thinking content: 
content: 下一步跟读的句子：`a says /æ/, /æ/ /æ/ apple`
thinking content: 
content: 下一步跟读的句子：  
**a says /æ/, /æ/ /æ/ apple**
thinking content: 
content: 下一步跟读的句子：`a says /æ/, /æ/ /æ/ apple`


In [10]:
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

model_my = AutoModelForCausalLM.from_pretrained(
    checkpoint_path
)

OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /root/group-shared/jrc/ppo-test/models/4gpu_aixue_data_test/checkpoint-1-out.